In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.io import fits
from tqdm.auto import tqdm 
import numpy as np
import os
import gc 
from glob import glob 
import sys
sys.path.append('../')
from modules import dendro_cutouts, dendro_misc

In [2]:
# Define names and filenames...

galaxy = 'ngc2835'
galaxy_hst = 'ngc2835s'
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = True
rerun_regions = True
rerun_masking = True

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/nebulae_catalogue/Nebulae_catalogue_v3.fits' %root_dir

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc2835s/hst_contsub/ngc2835s_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc2835s/muse/NGC2835_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc2835s/muse/NGC2835-1.15asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/dendro
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts_hdus
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/sample.reg
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/sample.pickel
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/nebulae_catal

In [3]:
# Make paths 

def make_paths(rootdir, galaxy_hst, rerun_all=False):

    root_dir = '%s/data_hstha_nebulae_catalogue/%s/' %(rootdir, galaxy_hst)
    cutout_dir = '%s/cutouts' %(root_dir)
    dendro_dir = '%s/dendro' %(root_dir)
    cutouts_hdus_dir = '%s/cutouts_hdus' %(root_dir)

    print('[Info] Outputing to the following:')
    print(root_dir)

    if rerun_all: 
        os.system('rm -rf '+root_dir)

    if not os.path.isdir(root_dir):
        os.mkdir(root_dir) 

    for path in [cutout_dir, dendro_dir, cutouts_hdus_dir]:

        if not os.path.isdir(path):
            os.mkdir(path)  

make_paths(root_dir, galaxy_hst)

[Info] Outputing to the following:
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/


In [4]:
# Loading files...

hstha_hdu = fits.open(hstha_file)[0]
musha_hdu = fits.open(musha_file)['HA6562_FLUX']
muscat_hdu = fits.open(muscat_file)[0]

# Update arrays
musha_hdu.data[np.isnan(musha_hdu.data)] = -100
muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

# Convert all hdus to float32
hdus = [hstha_hdu, musha_hdu, muscat_hdu]
hdus = [dendro_misc.convert_to_float32(hdu.copy()) for hdu in hdus]

In [5]:
# Get regions and sample table... 

if os.path.exists(regions_file) & ~rerun_regions:
    regions = dendro_misc.load_pickle(regions_pickel_file)
else: 
    muscat_table = dendro_misc.get_museprops(galaxy, muscat_table_file)
    regions_sky = dendro_cutouts.get_ds9regions_all(muscat_table, outputfile=regions_file)
    regions = dendro_cutouts.get_regions(regions_file)
    dendro_misc.save_pickle(regions, regions_pickel_file) 

[INFO] [get_MuseProps] Getting MUSE catalouge properties for ngc2835...
[INFO] [get_regions] Opening regions file (this may take a min)... 
[INFO] [get_regions] Getting info for 1121 regions...
[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/sample.pickel


In [6]:
# Get cutouts and save... 

hdus = [hstha_hdu, musha_hdu, muscat_hdu]
names = ['hstha_hdu', 'musha_hdu', 'muscat_hdu']

for hdu, name in zip(hdus, names):

    print('[INFO] Running for %s...' %name)
    hdu_cutouts = dendro_cutouts.get_croppeddata_all(hdu, regions)
    dendro_misc.save_pickle(hdu_cutouts, '%s/%s.pickel' %(cutout_dir, name)) 
    del hdu_cutouts
    _ = gc.collect()

[INFO] Running for hstha_hdu...


Cropping regions:   0%|          | 0/1121 [00:00<?, ?it/s]

[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 1
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 2
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 4
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 5
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 6
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 8
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 10
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 11
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 14
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 16
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 17
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 18
[INFO] [get_croppeddata] NoOverlapError: filling with nans for Region 19
[INFO] [get_croppeddata] NoOverlapError: filling with nan

Cropping regions:   0%|          | 0/1121 [00:00<?, ?it/s]

[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/musha_hdu.pickel
[INFO] Running for muscat_hdu...


Cropping regions:   0%|          | 0/1121 [00:00<?, ?it/s]

[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/muscat_hdu.pickel


In [7]:
# Load all cutouts and save in one file... 

names = ['hstha_hdu', 'musha_hdu', 'muscat_hdu']
hdus_cutouts = {}

for name in tqdm(names):

    hdu_cutouts = dendro_misc.load_pickle('%s/%s.pickel' %(cutout_dir, name))
    hdus_cutouts[name] = hdu_cutouts

dendro_misc.save_pickle(hdus_cutouts, '%s/hdus_all.pickel' %cutout_dir) 

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/hstha_hdu.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/musha_hdu.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/muscat_hdu.pickel
[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc2835s/cutouts/hdus_all.pickel
